In [2]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, auc, average_precision_score
#import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import keras.backend as K
from NGF.preprocessing import tensorise_smiles
from custom_layers.model_creator import encode_smiles, stage_creator
from keras.layers import Dense, Dropout, Input, Lambda, concatenate,Flatten
from keras.models import Model, load_model
import dill
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import xgboost as xgb
from math import ceil

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [3]:
def pairwise_distance(feature, squared):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums
def triplet_loss_adapted_from_tf(y_true, y_pred):
    del y_true
    margin = 1.0
    labels = y_pred[:, :1]
    labels = tf.cast(labels, dtype='int32')
    embeddings = y_pred[:, 1:]
    
    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=False)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

In [4]:
target_1 = 'p38'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)
    
target_2 = 'akt1'
base_path_2 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_2 = base_path_2+f'/data/{target_2}/data.csv'
df_akt1 = pd.read_csv(data_fpath_2).set_index('biolab_index')

with open(base_path_2+f'/data/{target_2}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_akt1 = dill.load(in_f)
with open(base_path_2+f'/data/{target_2}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_akt1 = dill.load(in_f)
    
target_3 = 'pi3k'
base_path_3 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_3 = base_path_3+f'/data/{target_3}/data.csv'
df_pi3k = pd.read_csv(data_fpath_3).set_index('biolab_index')

with open(base_path_3+f'/data/{target_3}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_pi3k = dill.load(in_f)
with open(base_path_3+f'/data/{target_3}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_pi3k = dill.load(in_f)

In [5]:
training_p38 = [df_p38.loc[train_val_folds_p38[0][0]],
                 df_p38.loc[train_val_folds_p38[1][0]],
                 df_p38.loc[train_val_folds_p38[2][0]],
                 df_p38.loc[train_val_folds_p38[3][0]],
                 df_p38.loc[train_val_folds_p38[4][0]],
                 df_p38.loc[train_val_folds_p38[5][0]],
                df_p38.loc[train_test_folds_p38[0]]
                 ]
validation_p38 = [df_p38.loc[train_val_folds_p38[0][1]],
                   df_p38.loc[train_val_folds_p38[1][1]],
                   df_p38.loc[train_val_folds_p38[2][1]],
                   df_p38.loc[train_val_folds_p38[3][1]],
                   df_p38.loc[train_val_folds_p38[4][1]],
                   df_p38.loc[train_val_folds_p38[5][1]],
                  df_p38.loc[train_test_folds_p38[1]]
                   ]

training_akt1 = [df_akt1.loc[train_val_folds_akt1[0][0]],
                 df_akt1.loc[train_val_folds_akt1[1][0]],
                 df_akt1.loc[train_val_folds_akt1[2][0]],
                 df_akt1.loc[train_val_folds_akt1[3][0]],
                 df_akt1.loc[train_val_folds_akt1[4][0]],
                 df_akt1.loc[train_val_folds_akt1[5][0]],
                 df_akt1.loc[train_test_folds_akt1[0]]
                 ]
validation_akt1 = [df_akt1.loc[train_val_folds_akt1[0][1]],
                   df_akt1.loc[train_val_folds_akt1[1][1]],
                   df_akt1.loc[train_val_folds_akt1[2][1]],
                   df_akt1.loc[train_val_folds_akt1[3][1]],
                   df_akt1.loc[train_val_folds_akt1[4][1]],
                   df_akt1.loc[train_val_folds_akt1[5][1]],
                   df_akt1.loc[train_test_folds_akt1[1]]
                   ]

training_pi3k = [df_pi3k.loc[train_val_folds_pi3k[0][0]],
                 df_pi3k.loc[train_val_folds_pi3k[1][0]],
                 df_pi3k.loc[train_val_folds_pi3k[2][0]],
                 df_pi3k.loc[train_val_folds_pi3k[3][0]],
                 df_pi3k.loc[train_val_folds_pi3k[4][0]],
                 df_pi3k.loc[train_val_folds_pi3k[5][0]],
                 df_pi3k.loc[train_test_folds_pi3k[0]]
                 ]
validation_pi3k = [df_pi3k.loc[train_val_folds_pi3k[0][1]],
                   df_pi3k.loc[train_val_folds_pi3k[1][1]],
                   df_pi3k.loc[train_val_folds_pi3k[2][1]],
                   df_pi3k.loc[train_val_folds_pi3k[3][1]],
                   df_pi3k.loc[train_val_folds_pi3k[4][1]],
                   df_pi3k.loc[train_val_folds_pi3k[5][1]],
                   df_pi3k.loc[train_test_folds_pi3k[1]]
                   ]

In [6]:
def csv_generator_mining(bs,df_p1,df_p2,df_p3,
                         Y_dummy_p1,Y_dummy_p2,Y_dummy_p3,
                         smiles_list_p1,smiles_list_p2,smiles_list_p3,
                         Y_p1,Y_p2,Y_p3,
                         atoms_p1,bonds_p1,edges_p1,
                         atoms_p2,bonds_p2,edges_p2,
                         atoms_p3,bonds_p3,edges_p3,mode = 'train',aug = None):
    
    counter_1=int(0)
    counter_2=int(0)
    counter_3=int(0)
    #Keep looping indefinetely
    df_p1 = df_p1.reset_index(drop=True)
    df_p2 = df_p2.reset_index(drop=True)
    df_p3 = df_p3.reset_index(drop=True)
    Y_p1 = np.array(list(Y_p1))
    Y_p2 = np.array(list(Y_p2))
    Y_p3 = np.array(list(Y_p3))
    while True:
        
        #Initialize batches of inputs and outputs
        ind1 = []
        ind2 = []
        ind3 = []
        
        d_p1=[]
        d_p2=[]
        d_p3=[]
        
        #Keep looping until we reach batch size
        while len(ind3)<=bs: #doesn't matter if it is smi1 or smi2 since they have the same len
            if counter_3==len(df_p3):
                counter_3=int(0)
                df_p3 = df_p3.sample(frac=1).reset_index(drop=True)
            
            smi_p3 = df_p3['rdkit'][counter_3]
            ind3.append(smiles_list_p3.index(smi_p3))
            d_p3.append(Y_dummy_p3[counter_3])
            counter_3+=1
            while len(ind1)<=bs:
                if counter_1==len(df_p1):
                    counter_1=int(0)
                    df_p1 = df_p1.sample(frac=1).reset_index(drop=True)
                
                smi_p1 = df_p1['rdkit'][counter_1]
                ind1.append(smiles_list_p1.index(smi_p1))
                d_p1.append(Y_dummy_p1[counter_1])
                counter_1+=1
                
                
                while len(ind2)<=bs:
            # check to see if you reached the end of the frame
                    if counter_2==len(df_p2):
                        counter_2=int(0)
                        df_p2 = df_p2.sample(frac=1).reset_index(drop=True)

                    smi_p2 = df_p2['rdkit'][counter_2]
                    ind2.append(smiles_list_p2.index(smi_p2))
                    d_p2.append(Y_dummy_p2[counter_2])
                    counter_2+=1
                # if we are evaluating we should now break from our 
                # loop to ensure we don't continue to fill up the batch from samples at the beginning of the file
                
                   
            
            

            #smi_p1 = df_p1['rdkit'][counter]
            #smi_p2 = df_p2['rdkit'][counter]
            #smi_p3 = df_p3['rdkit'][counter]
            #ind1.append(smiles_list_p1.index(smi_p1))
            #ind2.append(smiles_list_p2.index(smi_p2))
            #ind3.append(smiles_list_p3.index(smi_p3))
            #d_p1.append(Y_dummy_p1[counter])
            #d_p2.append(Y_dummy_p2[counter])
            #d_p3.append(Y_dummy_p3[counter])
            #counter+=1
            
        atoms_target_1 = np.array(atoms_p1[ind1],dtype = 'float32')
        bonds_target_1 = np.array(bonds_p1[ind1],dtype = 'float32')
        edges_target_1 = np.array(edges_p1[ind1],dtype = 'int32')
        labels_target_1 = Y_p1[ind1]
        
        atoms_target_2 = np.array(atoms_p2[ind2],dtype = 'float32')
        bonds_target_2 = np.array(bonds_p2[ind2],dtype = 'float32')
        edges_target_2 = np.array(edges_p2[ind2],dtype = 'int32')
        labels_target_2 = Y_p2[ind2]
        
        atoms_target_3 = np.array(atoms_p3[ind3],dtype = 'float32')
        bonds_target_3 = np.array(bonds_p3[ind3],dtype = 'float32')
        edges_target_3 = np.array(edges_p3[ind3],dtype = 'int32')
        labels_target_3 = Y_p3[ind3]
        
        yield ({'atom_inputs_1':atoms_target_1,
                'bond_inputs_1':bonds_target_1,
                'edge_inputs_1':edges_target_1,
                'labels_inputs_1':labels_target_1,
                'atom_inputs_2':atoms_target_2,
                'bond_inputs_2':bonds_target_2,
                'edge_inputs_2':edges_target_2,
                'labels_inputs_2':labels_target_2,
                'atom_inputs_3':atoms_target_3,
                'bond_inputs_3':bonds_target_3,
                'edge_inputs_3':edges_target_3,
                'labels_inputs_3':labels_target_3},{'Protein_1':np.array(d_p1,dtype = 'float32'),
                                                    'Protein_2':np.array(d_p2,dtype = 'float32'),
                                                    'Protein_3':np.array(d_p3,dtype = 'float32')})

In [7]:
def build_encoder(encoder_params):
        model_enc_1 = stage_creator(encoder_params, 1, conv=True)[0]
        model_enc_2 = stage_creator(encoder_params, 2, conv=True)[0]
        model_enc_3 = stage_creator(encoder_params, 3, conv=True)[0]

        model_enc_fp_1 = stage_creator(encoder_params, 1, conv=False)[1]
        model_enc_fp_2 = stage_creator(encoder_params, 2, conv=False)[1]
        model_enc_fp_3 = stage_creator(encoder_params, 3, conv=False)[1]

        atoms, bonds, edges = encode_smiles(encoder_params["max_atoms"],
                                            encoder_params["num_atom_features"],
                                            encoder_params["max_degree"],
                                            encoder_params["num_bond_features"])

        graph_conv_1 = model_enc_1([atoms, bonds, edges])
        graph_conv_2 = model_enc_2([graph_conv_1, bonds, edges])
        graph_conv_3 = model_enc_3([graph_conv_2, bonds, edges])

        fingerprint_1 = model_enc_fp_1([graph_conv_1, bonds, edges])
        fingerprint_1 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_1)

        fingerprint_2 = model_enc_fp_2([graph_conv_2, bonds, edges])
        fingerprint_2 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_2)

        fingerprint_3 = model_enc_fp_3([graph_conv_3, bonds, edges])
        fingerprint_3 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_3)

        final_fingerprint = keras.layers.add([fingerprint_1, fingerprint_2, fingerprint_3])

        return Model([atoms, bonds, edges], [final_fingerprint])

In [8]:
def build_model(model_params, encoder, verbose=False):
        atoms_1 = Input(name='atom_inputs_1',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds_1 = Input(name='bond_inputs_1', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges_1 = Input(name='edge_inputs_1', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug_1 = encoder([atoms_1, bonds_1, edges_1])
        
        atoms_2 = Input(name='atom_inputs_2',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds_2 = Input(name='bond_inputs_2', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges_2 = Input(name='edge_inputs_2', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug_2 = encoder([atoms_2, bonds_2, edges_2])
        
        atoms_3 = Input(name='atom_inputs_3',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds_3 = Input(name='bond_inputs_3', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges_3 = Input(name='edge_inputs_3', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug_3 = encoder([atoms_3, bonds_3, edges_3])
        
        conc = keras.layers.Concatenate(axis = -1)([encode_drug_1,encode_drug_2,encode_drug_3])
        # Fully connected
        FC1 = Dense(model_params["dense_size"][0], activation='relu',kernel_initializer='random_normal')(conc)
        FC2 = Dropout(model_params["dropout_rate"][0])(FC1)
        FC2 = Dense(model_params["dense_size"][1], activation='relu',kernel_initializer='random_normal')(FC2)
        FC2 = Dropout(model_params["dropout_rate"][1])(FC2)
        FC2 = Dense(model_params["dense_size"][2], activation = None,kernel_initializer='random_normal')(FC2)
        
        
        embeddings_conc = Lambda(lambda x: K.l2_normalize(x,axis=1),name = 'Embeddings_Concatenated')(FC2)
        embeddings_1 = crop(1,0,int(model_params['dense_size'][2]/3),name = 'Target_1')(embeddings_conc) #0 to 255(256)
        embeddings_2 = crop(1,int(model_params['dense_size'][2]/3),2*int(model_params['dense_size'][2]/3),name = 'Target_2')(embeddings_conc) #256 to 511(256)
        embeddings_3 = crop(1,2*int(model_params['dense_size'][2]/3),3*int(model_params['dense_size'][2]/3),name = 'Target_3')(embeddings_conc)
        
        gcn_model = Model(inputs=[atoms_1, bonds_1, edges_1,
                                  atoms_2, bonds_2, edges_2,
                                  atoms_3, bonds_3, edges_3], 
                                  outputs = [embeddings_1,
                                             embeddings_2,
                                             embeddings_3]
                         )


        if verbose:
            #print('encoder')
            #encoder.summary()
            print('GCN_model')
            gcn_model.summary()
        
            
        return gcn_model

In [9]:
def build_mining(model_params,gcn_model):
        #First Target
        atoms_1 = Input(name='atom_inputs_1',shape=(model_params['max_atoms'], 
                                                    model_params['num_atom_features']), 
                                                    dtype='float32')
        bonds_1 = Input(name='bond_inputs_1', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree'], 
                                                     model_params['num_bond_features']),
                                                     dtype='float32')
        edges_1 = Input(name='edge_inputs_1', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree']),
                                                     dtype='int32')
        
        labels_1 = Input(name = 'labels_inputs_1',shape = (1,),dtype = 'float32')
        
        #Second Target
        atoms_2 = Input(name='atom_inputs_2',shape=(model_params['max_atoms'], 
                                                    model_params['num_atom_features']), 
                                                    dtype='float32')
        bonds_2 = Input(name='bond_inputs_2', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree'], 
                                                     model_params['num_bond_features']),
                                                     dtype='float32')
        edges_2 = Input(name='edge_inputs_2', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree']),
                                                     dtype='int32')
        
        labels_2 = Input(name = 'labels_inputs_2',shape = (1,),dtype = 'float32')
        
        #Third Target
        atoms_3 = Input(name='atom_inputs_3',shape=(model_params['max_atoms'], 
                                                    model_params['num_atom_features']), 
                                                    dtype='float32')
        bonds_3 = Input(name='bond_inputs_3', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree'], 
                                                     model_params['num_bond_features']),
                                                     dtype='float32')
        edges_3 = Input(name='edge_inputs_3', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        
        labels_3 = Input(name = 'labels_inputs_3',shape = (1,),dtype = 'float32')
        encoded_1,encoded_2,encoded_3 = gcn_model([atoms_1,bonds_1,edges_1,
                                                   atoms_2,bonds_2,edges_2,
                                                   atoms_3,bonds_3,edges_3])
        
        labels_plus_embeddings_1 = concatenate([labels_1, encoded_1],name = 'Protein_1')
        labels_plus_embeddings_2 = concatenate([labels_2, encoded_2],name = 'Protein_2')
        labels_plus_embeddings_3 = concatenate([labels_3, encoded_3],name = 'Protein_3')
        
        mining_net = Model(inputs = [atoms_1,bonds_1,edges_1,labels_1,
                                     atoms_2,bonds_2,edges_2,labels_2,
                                     atoms_3,bonds_3,edges_3,labels_3],
                           outputs = [labels_plus_embeddings_1,
                                      labels_plus_embeddings_2,
                                      labels_plus_embeddings_3])
        adam = keras.optimizers.Adam(lr = model_params["lr"], 
                                     beta_1=0.9, 
                                     beta_2=0.999, 
                                     decay=0.0, 
                                     amsgrad=False)
    
    
        mining_net.compile(optimizer=adam , loss = {'Protein_1' : triplet_loss_adapted_from_tf,
                                                    'Protein_2' : triplet_loss_adapted_from_tf,
                                                    'Protein_3' : triplet_loss_adapted_from_tf})
        mining_net.summary()
        return mining_net

In [10]:
def dataframe_to_gcn_input(input_data):
        x_atoms_cold, x_bonds_cold, x_edges_cold = tensorise_smiles(input_data['rdkit'],
                                                                    max_degree=model_params['max_degree'],
                                                                    max_atoms=model_params['max_atoms'])
        return [x_atoms_cold, x_bonds_cold, x_edges_cold]

In [11]:
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(80), int(100), int(160)], #[int(56), int(88), int(136)],
        "fp_length" : [int(152.0), int(152.0), int(152.0)],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(3*300), int(3*200), int(3*100)], 
        'dropout_rate' : [0.27225175676555935, 0.27225175676555935],
        'lr' : 0.0008110012706176706,
        'batch_size' : int(288.0),
        'n_epochs' : int(30),
        'margin' : 1
        }
xgb_hyper = {
        "colsample_bylevel" : 0.4371082812232264,
        "colsample_bytree" : 0.4179415558635843,
        "gamma" : 0.919836526180396,
        "eta" : 0.41409388868400826,
        "max_delta_step" : int(2),
        "max_depth" : int(7),
        "min_child_weight" : int(20),
        "alpha" : 0.15030685758880047,
        "lambda" : 12.306130216692438,
        "subsample" : 0.6038298323514097,
        "max_bin" : int(48.0),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree',
        #"single_precision_histogram" : True
}

In [12]:
def crop(dimension, start, end,name):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func,name = name)

In [13]:
train_p1 = {}
train_p2 = {}
train_p3 = {}
val_p1 = {}
val_p2 = {}
val_p3 = {}
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.000000001)
for i in range(0,1):#range(len(training_p38)):
    
        #First Target
        X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38 = dataframe_to_gcn_input(validation_p38[i])
        Y_cold_p38 = validation_p38[i].Binary
        Y_dummy_cold_p38 = np.empty((X_atoms_cold_p38.shape[0],int(model_params['dense_size'][2]/3)+1))
        X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38 = dataframe_to_gcn_input(training_p38[i])
        Y_p38 = training_p38[i].Binary
        Y_dummy_train_p38 = np.empty((X_atoms_train_p38.shape[0],int(model_params['dense_size'][2]/3)+1))
        smiles_list_p38 = list(training_p38[i]['rdkit'])
        
        #Second Target
        X_atoms_cold_akt1, X_bonds_cold_akt1, X_edges_cold_akt1 = dataframe_to_gcn_input(validation_akt1[i])
        Y_cold_akt1 = validation_akt1[i].Binary
        Y_dummy_cold_akt1 = np.empty((X_atoms_cold_akt1.shape[0],int(model_params['dense_size'][2]/3)+1))
        X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1 = dataframe_to_gcn_input(training_akt1[i])
        Y_akt1 = training_akt1[i].Binary
        Y_dummy_train_akt1 = np.empty((X_atoms_train_akt1.shape[0],int(model_params['dense_size'][2]/3)+1))
        smiles_list_akt1 = list(training_akt1[i]['rdkit'])
        
        #Third Target
        X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k = dataframe_to_gcn_input(validation_pi3k[i])
        Y_cold_pi3k = validation_pi3k[i].Binary
        Y_dummy_cold_pi3k = np.empty((X_atoms_cold_pi3k.shape[0],int(model_params['dense_size'][2]/3)+1))
        X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k = dataframe_to_gcn_input(training_pi3k[i])
        Y_pi3k = training_pi3k[i].Binary
        Y_dummy_train_pi3k = np.empty((X_atoms_train_pi3k.shape[0],int(model_params['dense_size'][2]/3)+1))
        smiles_list_pi3k = list(training_pi3k[i]['rdkit'])
        
        gcn_encoder = build_encoder(model_params)
        gcn_model = build_model(model_params,gcn_encoder)
        gcn_mining = build_mining(model_params,gcn_model)
        train_GEN = csv_generator_mining(model_params['batch_size'],training_p38[i],training_akt1[i],training_pi3k[i],
                                  Y_dummy_train_p38,Y_dummy_train_akt1,Y_dummy_train_pi3k,
                                  smiles_list_p38,smiles_list_akt1,smiles_list_pi3k,
                                  Y_p38,Y_akt1,Y_pi3k,
                                  X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38,
                                  X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1,
                                  X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k)
        NUM_TRAIN = np.sum([len(training_p38[i]),len(training_akt1[i]),len(training_pi3k[i])])
        gcn_mining.fit_generator(train_GEN,
                      steps_per_epoch = ceil(2*NUM_TRAIN/model_params['batch_size']),
                      epochs = model_params['n_epochs'],
                      shuffle = True,
                      validation_data = None,
                      callbacks = [es2,rlr2])
        emb_train_1,t1,t2 = gcn_model.predict([X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38,
                                        X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38,
                                        X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38])
        t1,emb_train_2,t2 = gcn_model.predict([X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1,
                                        X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1,
                                        X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1])
        t1,t2,emb_train_3 = gcn_model.predict([X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k,
                                        X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k,
                                        X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k])


        emb_val_1,t1,t2 = gcn_model.predict([X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38,
                                            X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38,
                                            X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38])
        t1,emb_val_2,t2 = gcn_model.predict([X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1,
                                            X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1,
                                            X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1])
        t1,t2,emb_val_3 = gcn_model.predict([X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k,
                                            X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k,
                                            X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k])
        
        del t1,t2, gcn_mining,gcn_encoder,gcn_model,X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38
        del X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38,X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k
        del X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k,X_atoms_cold_akt1, X_bonds_cold_akt1, X_edges_cold_akt1
        del X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1,train_GEN
        
        #First
        dmatrix_train_1 = xgb.DMatrix(data = emb_train_1,label = Y_p38)
        dmatrix_cold_1  = xgb.DMatrix(data = emb_val_1,label = Y_cold_p38)
        evalist_1 = [(dmatrix_train_1,'train'),(dmatrix_cold_1,'eval')]
        xgb_model_1 = xgb.train(xgb_hyper,dmatrix_train_1,300,evalist_1,verbose_eval=True)
        xgb_pred_cold_1 = xgb_model_1.predict(dmatrix_cold_1)
        xgb_pred_train_1 = xgb_model_1.predict(dmatrix_train_1)
        
        #Second
        dmatrix_train_2 = xgb.DMatrix(data = emb_train_2,label = Y_akt1)
        dmatrix_cold_2  = xgb.DMatrix(data = emb_val_2,label = Y_cold_akt1)
        evalist_2 = [(dmatrix_train_2,'train'),(dmatrix_cold_2,'eval')]
        xgb_model_2 = xgb.train(xgb_hyper,dmatrix_train_2,300,evalist_2,verbose_eval=True)
        xgb_pred_cold_2 = xgb_model_2.predict(dmatrix_cold_2)
        xgb_pred_train_2 = xgb_model_2.predict(dmatrix_train_2)
        
        #Third
        dmatrix_train_3 = xgb.DMatrix(data = emb_train_3,label = Y_pi3k)
        dmatrix_cold_3  = xgb.DMatrix(data = emb_val_3,label = Y_cold_pi3k)
        evalist_3 = [(dmatrix_train_3,'train'),(dmatrix_cold_3,'eval')]
        xgb_model_3 = xgb.train(xgb_hyper,dmatrix_train_3,300,evalist_3,verbose_eval=True)
        xgb_pred_cold_3 = xgb_model_3.predict(dmatrix_cold_3)
        xgb_pred_train_3 = xgb_model_3.predict(dmatrix_train_3)
        
        if i < 6:
            val_p1['Val_%s'%i] = calculate_metrics(np.array(Y_cold_p38),xgb_pred_cold_1)
            val_p2['Val_%s'%i] = calculate_metrics(np.array(Y_cold_akt1),xgb_pred_cold_2)
            val_p3['Val_%s'%i] = calculate_metrics(np.array(Y_cold_pi3k),xgb_pred_cold_3)
            
            train_p1['Fold_%s'%i] = calculate_metrics(np.array(Y_p38),xgb_pred_train_1)
            train_p2['Fold_%s'%i] = calculate_metrics(np.array(Y_akt1),xgb_pred_train_2)
            train_p3['Fold_%s'%i] = calculate_metrics(np.array(Y_pi3k),xgb_pred_train_3)
        elif i == 6:
            val_p1['Test'] = calculate_metrics(np.array(Y_cold_p38),xgb_pred_cold_1)
            val_p2['Test'] = calculate_metrics(np.array(Y_cold_akt1),xgb_pred_cold_2)
            val_p3['Test'] = calculate_metrics(np.array(Y_cold_pi3k),xgb_pred_cold_3)
            
            train_p1['Test'] = calculate_metrics(np.array(Y_p38),xgb_pred_train_1)
            train_p2['Test'] = calculate_metrics(np.array(Y_akt1),xgb_pred_train_2)
            train_p3['Test'] = calculate_metrics(np.array(Y_pi3k),xgb_pred_train_3)

LAYER 0
LAYER 1
LAYER 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs_1 (InputLayer)      (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs_1 (InputLayer)      (None, 70, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs_1 (InputLayer)      (None, 70, 5)        0                                            
__________________________________________________________________________________________________
atom_inputs_2 (InputLayer)      (None, 70, 62)       0                                            
_____________________________________________________________________________________

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30
47/47 [==============================] - 18s 381ms/step - loss: 2.9947 - Protein_1_loss: 0.9984 - Protein_2_loss: 0.9982 - Protein_3_loss: 0.9981
Epoch 2/30
47/47 [==============================] - 10s 202ms/step - loss: 2.9888 - Protein_1_loss: 0.9967 - Protein_2_loss: 0.9956 - Protein_3_loss: 0.9965
Epoch 3/30
47/47 [==============================] - 9s 202ms/step - loss: 2.9738 - Protein_1_loss: 0.9946 - Protein_2_loss: 0.9862 - Protein_3_loss: 0.9930
Epoch 4/30
47/47 [==============================] - 10s 203ms/step - loss: 2.9656 - Protein_1_loss: 0.9953 - Protein_2_loss: 0.9817 - Protein_3_loss: 0.9887
Epoch 5/30
47/47 [==============================] - 10s 203ms/step - loss: 2.9577 - Protein_1_loss: 0.9960 - Protein_2_loss: 0.9747 - Protein_3_loss: 0.9870
Epoch 6/30
47/47 [==============================] - 10s 203ms/step - loss: 2.9563 - Protein_1_loss: 0.9956 - Protein_2_loss: 0.9736 - Protein_3_loss: 0.9872
Epoch 7/30
47/47 [==============================] - 10s 203

[83]	train-auc:0.89410	eval-auc:0.62104
[84]	train-auc:0.89673	eval-auc:0.62454
[85]	train-auc:0.89731	eval-auc:0.62309
[86]	train-auc:0.89986	eval-auc:0.62929
[87]	train-auc:0.90041	eval-auc:0.62850
[88]	train-auc:0.90140	eval-auc:0.62676
[89]	train-auc:0.90024	eval-auc:0.62514
[90]	train-auc:0.90111	eval-auc:0.62701
[91]	train-auc:0.90228	eval-auc:0.62463
[92]	train-auc:0.90595	eval-auc:0.62300
[93]	train-auc:0.90703	eval-auc:0.62651
[94]	train-auc:0.90709	eval-auc:0.62486
[95]	train-auc:0.90833	eval-auc:0.62336
[96]	train-auc:0.90863	eval-auc:0.62541
[97]	train-auc:0.90835	eval-auc:0.62732
[98]	train-auc:0.91216	eval-auc:0.62631
[99]	train-auc:0.91140	eval-auc:0.62575
[100]	train-auc:0.91245	eval-auc:0.62146
[101]	train-auc:0.91342	eval-auc:0.61941
[102]	train-auc:0.91384	eval-auc:0.62039
[103]	train-auc:0.91383	eval-auc:0.62493
[104]	train-auc:0.91373	eval-auc:0.62929
[105]	train-auc:0.91590	eval-auc:0.62826
[106]	train-auc:0.91943	eval-auc:0.62241
[107]	train-auc:0.91953	eval-auc:

[284]	train-auc:0.97994	eval-auc:0.58317
[285]	train-auc:0.98006	eval-auc:0.58776
[286]	train-auc:0.98032	eval-auc:0.58657
[287]	train-auc:0.98077	eval-auc:0.58846
[288]	train-auc:0.98049	eval-auc:0.58876
[289]	train-auc:0.98058	eval-auc:0.58475
[290]	train-auc:0.98078	eval-auc:0.58276
[291]	train-auc:0.98104	eval-auc:0.58045
[292]	train-auc:0.98100	eval-auc:0.58000
[293]	train-auc:0.98058	eval-auc:0.58037
[294]	train-auc:0.98096	eval-auc:0.57712
[295]	train-auc:0.98142	eval-auc:0.57879
[296]	train-auc:0.98129	eval-auc:0.58085
[297]	train-auc:0.98139	eval-auc:0.58261
[298]	train-auc:0.98115	eval-auc:0.57919
[299]	train-auc:0.98165	eval-auc:0.57821
[0]	train-auc:0.94955	eval-auc:0.81528
[1]	train-auc:0.96709	eval-auc:0.81639
[2]	train-auc:0.97361	eval-auc:0.84136
[3]	train-auc:0.97647	eval-auc:0.85069
[4]	train-auc:0.97724	eval-auc:0.85217
[5]	train-auc:0.97871	eval-auc:0.85691
[6]	train-auc:0.97890	eval-auc:0.85640
[7]	train-auc:0.97912	eval-auc:0.85648
[8]	train-auc:0.97934	eval-auc:0

[187]	train-auc:0.98940	eval-auc:0.86592
[188]	train-auc:0.98936	eval-auc:0.86479
[189]	train-auc:0.98924	eval-auc:0.86464
[190]	train-auc:0.98937	eval-auc:0.86272
[191]	train-auc:0.98951	eval-auc:0.85853
[192]	train-auc:0.98938	eval-auc:0.85909
[193]	train-auc:0.98932	eval-auc:0.85943
[194]	train-auc:0.98932	eval-auc:0.85943
[195]	train-auc:0.98915	eval-auc:0.85998
[196]	train-auc:0.98915	eval-auc:0.85998
[197]	train-auc:0.98940	eval-auc:0.85986
[198]	train-auc:0.98935	eval-auc:0.86216
[199]	train-auc:0.98956	eval-auc:0.86220
[200]	train-auc:0.98975	eval-auc:0.86161
[201]	train-auc:0.99001	eval-auc:0.86131
[202]	train-auc:0.99014	eval-auc:0.86050
[203]	train-auc:0.99014	eval-auc:0.86050
[204]	train-auc:0.99006	eval-auc:0.86097
[205]	train-auc:0.98986	eval-auc:0.86114
[206]	train-auc:0.98986	eval-auc:0.86114
[207]	train-auc:0.99002	eval-auc:0.86118
[208]	train-auc:0.99002	eval-auc:0.86118
[209]	train-auc:0.99002	eval-auc:0.86118
[210]	train-auc:0.98982	eval-auc:0.86139
[211]	train-auc:

[90]	train-auc:0.93243	eval-auc:0.72416
[91]	train-auc:0.93240	eval-auc:0.72387
[92]	train-auc:0.93384	eval-auc:0.72221
[93]	train-auc:0.93471	eval-auc:0.72248
[94]	train-auc:0.93549	eval-auc:0.72412
[95]	train-auc:0.93511	eval-auc:0.72430
[96]	train-auc:0.93576	eval-auc:0.72180
[97]	train-auc:0.93611	eval-auc:0.72140
[98]	train-auc:0.93687	eval-auc:0.71999
[99]	train-auc:0.93717	eval-auc:0.72156
[100]	train-auc:0.93685	eval-auc:0.72223
[101]	train-auc:0.93626	eval-auc:0.72207
[102]	train-auc:0.93704	eval-auc:0.72294
[103]	train-auc:0.93835	eval-auc:0.72335
[104]	train-auc:0.93918	eval-auc:0.72332
[105]	train-auc:0.93999	eval-auc:0.72385
[106]	train-auc:0.94020	eval-auc:0.72440
[107]	train-auc:0.94084	eval-auc:0.72443
[108]	train-auc:0.94091	eval-auc:0.72619
[109]	train-auc:0.94190	eval-auc:0.72429
[110]	train-auc:0.94230	eval-auc:0.72560
[111]	train-auc:0.94236	eval-auc:0.72384
[112]	train-auc:0.94284	eval-auc:0.72329
[113]	train-auc:0.94215	eval-auc:0.72260
[114]	train-auc:0.94329	ev

[291]	train-auc:0.97993	eval-auc:0.72115
[292]	train-auc:0.98039	eval-auc:0.72055
[293]	train-auc:0.98010	eval-auc:0.72088
[294]	train-auc:0.98035	eval-auc:0.72011
[295]	train-auc:0.98031	eval-auc:0.72134
[296]	train-auc:0.98028	eval-auc:0.72136
[297]	train-auc:0.98032	eval-auc:0.72210
[298]	train-auc:0.98069	eval-auc:0.72194
[299]	train-auc:0.98070	eval-auc:0.72200


In [14]:
pd.DataFrame(val_p2).T

,accuracy,fn,fp,map,precision,recall,roc_auc,tn,tp
Val_0,0.777778,24.0,44.0,0.865027,0.747126,0.844156,0.865751,108.0,130.0


In [19]:
pd.DataFrame(train_p2).T

,accuracy,fn,fp,map,precision,recall,roc_auc,tn,tp
Fold_0,0.947644,37.0,43.0,0.980781,0.930421,0.939542,0.987977,873.0,575.0
